### Importing the essential libraries first
First step is to import the essential libraries that will be used for the processing of data.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Reading the Dataset
I have been using amazon_baby data which has reviews of people with their name and rating.

In [5]:
products=pd.read_csv('amazon_baby.csv')
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


### Droping the rows with the null values
Now we are going to drop the rows having atleast one null values. We have 1147 rows with null values which has been removed from the dataset

In [6]:
len(products)-len(products.dropna())

1147

In [7]:
products=products.dropna()

### Checking that any review data has string type
We are checking if any review column has string data type. If not then we are converting it into string datatype so that no problem occurs afterwards.

In [8]:
for i in range(0,len(products)-1):
    if type(products.iloc[i]['review']) != str:
        products.iloc[i]['review'] = str(products.iloc[i]['review'])

### Telling our model which rating is positive and which is negative
I am going to tell my model that the rating 1,2 are negative and rating 4,5 are good. Rating 3 is in the middle so its not effecting the model decsion so I am removing it.

In [9]:
products=products[products['rating']!=3]

In [10]:
def sentiment(n):
    return 1 if n>=4 else 0
products['sentiment']=products['rating'].apply(sentiment)
products.head()

,name,review,rating,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,1


### Combining the name and review table
We are going to combine name and the review columns to make a single column.

In [11]:
def combined_feature(row):
    return row['name']+ ' ' +row['review']


In [12]:
products['all_features']=products.apply(combined_feature,axis=1)
products.head()

,name,review,rating,sentiment,all_features
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,1,Planetwise Wipe Pouch it came early and was no...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1,Annas Dream Full Quilt with 2 Shams Very soft ...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,1,Stop Pacifier Sucking without tears with Thumb...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,Stop Pacifier Sucking without tears with Thumb...
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,1,Stop Pacifier Sucking without tears with Thumb...


### Getting the independent and dependent variables

In [13]:
X=products['all_features']
Y=products['sentiment']

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=0)

In [15]:
X_train

115613    Lamaze Cloth Book, Panda\'s Pals I used this b...
44536     Munchkin Arm and Hammer Bag Refill, 36 Bags I ...
167231    Samsung SEW-3037W Wireless Pan Tilt Video Baby...
87182     Medela Easy Expression Hands-Free Bustier, Whi...
7520      The First Year\'s Infant To Toddler Tub with S...
                                ...                        
108099    Bright Starts Bounce Baby Activity Zone, Neutr...
106237    NUK Ultra Thin Breast Pads, Pack of 2, White, ...
168698    Graco FastAction Fold Jogger Click Connect Str...
130592    Serta Deluxe Waterproof Mattress Cover This cr...
48275     Munchkin 36 Bath Letters and Numbers First col...
Name: all_features, Length: 124259, dtype: object

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
ctmTr=cv.fit_transform(X_train)
X_test_dtm=cv.transform(X_test)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression()
model.fit(ctmTr, y_train)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
y_pred_class = model.predict(X_test_dtm)
y_pred_class

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [21]:
accuracy_score(y_test, y_pred_class)


0.9323756639304683